## 设计和实现可复用的生成器

目前已经基于生成器实现了各个功能，但仔细分析可以发现一些重复

- 数列前N项（基于下标的判断）
- 数列第N至M项（基于下标的判断）
- 值为奇数的N项（基于值的判断）
- 下标为奇数的N项（基于下标的判断）

于是可以实现两个高度抽象的筛选函数

- 基于下标的筛选
- 基于值的筛选

In [ ]:
def generate_fib():
  yield 1
  yield 1
  last_two = [1, 1]
  while True:
    v = sum(last_two)
    yield v
    last_two = [last_two[-1], v]

In [ ]:
def filter_by_value(g, should_take):
  for v in g:
    if should_take(v):
      yield v


def filter_by_index(g, should_take):
  for idx, v in enumerate(g):
    if should_take(idx):
      yield v

这两个筛选函数实质上也是无限长的生成器。为了取得有限长的结果，还可以补充一个定长的筛选函数。

In [ ]:
def take_first_n(g, n):
  for idx, v in enumerate(g):
    if idx >= n:
      break
    yield v

### 组合

通过组合各个生成器并添加判定函数，可以满足各种需求

In [ ]:
def fib_first_n(n):
  return take_first_n(
    generate_fib(),
    n
  )


list(fib_first_n(10))

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

In [ ]:
def fib_n_to_m(n, m):
  return take_first_n(
    filter_by_index(generate_fib(), lambda idx: idx >= n),
    m - n,
  )


list(fib_n_to_m(5, 10))

[8, 13, 21, 34, 55]

In [ ]:
def fib_odd_value_n(n):
  return take_first_n(
    filter_by_value(generate_fib(), lambda v: v % 2 == 1),
    n,
  )


list(fib_odd_value_n(7))

[1, 1, 3, 5, 13, 21, 55]

In [ ]:
def fib_odd_index_n(n):
  return take_first_n(
    filter_by_index(generate_fib(), lambda idx: idx % 2 == 1),
    n,
  )


list(fib_odd_index_n(5))

[1, 3, 8, 21, 55]

## 直接套用工具

In [ ]:
import itertools

In [ ]:
list(
  itertools.islice(generate_fib(), 10)
)

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

In [ ]:
list(
  itertools.islice(generate_fib(), 5, 10)
)

[8, 13, 21, 34, 55]

In [ ]:
list(
  itertools.islice(
    filter(lambda v: v % 2 == 1, generate_fib()),
    7
  )
)

[1, 1, 3, 5, 13, 21, 55]

In [ ]:
list(
  itertools.islice(
    map(
      lambda idx_and_v: idx_and_v[1],
      filter(
        lambda idx_and_v: idx_and_v[0] % 2 == 1,
        enumerate(generate_fib())
      )
    ),
    5
  )
)

[1, 3, 8, 21, 55]

In [ ]:
list(
  itertools.islice(
    filter_by_index(
      generate_fib(),
      lambda idx: idx % 2 == 1,
    ),
    5
  )
)

[1, 3, 8, 21, 55]